In [1]:
#Import libraries 
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
data = pd.read_csv("merged_final.csv")

/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,13,29,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data2=data[['title','rating','user_id']] #Subset the data
data3=data2.groupby('title')[['rating','user_id']].agg('count') #Groupby the title to get rating and user id for each book
mask=data3['rating']>=10 #Remove any book with less than 10 total ratings
data4=data3[mask] #Create a dataframe with only book with greater than 10 ratings 
merged_inner=pd.merge(left=data4,right=data2,left_on='title',right_on='title',how='inner') #Inner merge of data2 and data4 
Final=merged_inner[['title','rating_y','user_id_y']] #Final data frame with the book title, rating, and user id 

In [3]:
#pivot ratings into book features
df_book_features = pd.pivot_table(data=Final,
    index='title',
    columns='user_id_y',
    values='rating_y',fill_value=0)
title_list=df_book_features.index.tolist()

In [4]:
#Creating a csr matrix 
mat_book_features = csr_matrix(df_book_features.values)

In [5]:
#Options for the KNN model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)

In [6]:
#Fitting the KNN model on the csr matrix 
model_knn.fit(mat_book_features)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [41]:
#User input of the last book (Or entry into search engine)
test_title=input("What book have you recently read: ")

What book have you recently read: Daredevil comic 


In [42]:
#Creating a list of titles to check input against 
query_index=title_list.index(process.extract(test_title,title_list)[0][0])

In [43]:
#Creating the output for a given input book using cosine distance
distances, indices = model_knn.kneighbors(df_book_features.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)
for i in range(0,len(distances.flatten())):
    if i ==0:
        print('Recommendations for {0}:\n'.format(df_book_features.index[query_index]))
    else:
        print('{0}:{1}, with distance of {2}.'.format(i,df_book_features.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendations for Red:

1:Impossible Views of the World, with distance of 0.6450573962335544.
2:A Strange Scottish Shore (Emmaline Truelove, #2), with distance of 0.6450573962335544.
3:Forbidden Fruit, with distance of 0.6705183593240245.
4:Rough Justice (Rosato & Associates, #3), with distance of 0.74384224021072.
5:Austentatious, with distance of 0.7490176779447366.


In [44]:
#Creating the output for a given input book using euclidean distance
model_knn2 = NearestNeighbors(metric='euclidean', algorithm='brute', n_neighbors=20, n_jobs=-1)
model_knn2.fit(mat_book_features)
distances, indices = model_knn2.kneighbors(df_book_features.iloc[query_index, :].values.reshape(1, -1), n_neighbors = 6)
for i in range(0,len(distances.flatten())):
    if i ==0:
        print('Recommendations for {0}:\n'.format(df_book_features.index[query_index]))
    else:
        print('{0}:{1}, with distance of {2}.'.format(i,df_book_features.index[indices.flatten()[i]],distances.flatten()[i]))

Recommendations for Red:

1:A Strange Scottish Shore (Emmaline Truelove, #2), with distance of 10.583005244258363.
2:Impossible Views of the World, with distance of 10.723805294763608.
3:Austentatious, with distance of 11.0.
4:Epic Big Nate, with distance of 11.269427669584644.
5:The Iceberg, with distance of 11.269427669584644.
